# 🎮 Quando Foi a Era de Ouro dos Videogames
 ### Projeto SQL do [DataCamp](https://www.datacamp.com/projects/1413)
 * Dataset original no [Kaggle](https://www.kaggle.com/holmjason2/videogamedata)
 <p>Tabelas utilizadas</p>
<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>Tipo</th>
<th>Significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Nome do video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Plataforma de jogos</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Publicadora</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Desenvolvedora</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Número de cópias vendidas (milhões)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Ano de lançamento</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>Tipo</th>
<th>Significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Nome do video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td>Nota da crítica de acordo com o  Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td>Nota de usúarios de acordo com o Metacritic</td>
</tr>
</tbody>
</table>

### 1. Os 10 jogos mais vendidos de todos os tempos.

In [158]:
%%sql
postgresql :/// games 
SELECT 
  * 
FROM 
  game_sales 
ORDER BY 
  games_sold DESC 
LIMIT 
  10;



10 rows affected.


game,platform,publisher,developer,games_sold,year
Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


### 2. Contando Reviews com valores nulos


In [160]:
%%sql
SELECT 
  COUNT(game_sales.game) 
FROM 
  game_sales 
  LEFT JOIN reviews ON game_sales.game = reviews.game 
WHERE 
  reviews.critic_score IS NULL 
  AND reviews.user_score IS NULL 
LIMIT 
  10;


 * postgresql:///games
1 rows affected.


count
31


### 3. Anos com melhores avaliações de críticos 
O número de valores nulos é pequeno então serão ignorados

In [162]:
%%sql
SELECT 
  g.year, 
  ROUND(
    AVG(r.critic_score), 
    2
  ) AS avg_critic_score 
FROM 
  game_sales g 
  INNER JOIN reviews r ON g.game = r.game 
GROUP BY 
  year 
ORDER BY 
  avg_critic_score DESC 
LIMIT 
  10;



 * postgresql:///games
10 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
1998,9.32
2020,9.20
1993,9.10
1995,9.07
2004,9.03
1982,9.00
2002,8.99
1999,8.93


### 4. Selecionando média apenas de ano que possuem mais 4 jogas lançados


In [164]:
%%sql 
SELECT 
  g.year, 
  COUNT(g.game) AS num_games, 
  ROUND(
    AVG(r.critic_score), 
    2
  ) AS avg_critic_score 
FROM 
  game_sales g 
  INNER JOIN reviews r ON g.game = r.game 
GROUP BY 
  g.year 
HAVING 
  COUNT(g.game) > 4 
ORDER BY 
  avg_critic_score DESC 
LIMIT 
  10;



 * postgresql:///games
10 rows affected.


year,num_games,avg_critic_score
1998,10,9.32
2004,11,9.03
2002,9,8.99
1999,11,8.93
2001,13,8.82
2011,26,8.76
2016,13,8.67
2013,18,8.66
2008,20,8.63
2017,13,8.62


## 5. Anos que ficaram de fora da lista por terem menos de 4 jogos lançados
Foram criadas duas tabelas com o resultado das consultas anteriores:</p>
<h3 id="top_critic_years"><code>top_critic_years</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>Tipo</th>
<th>Significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Ano de lançamento</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Média de todas as notas de críticos por ano</td>
</tr>
</tbody>
</table>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>Tipo</th>
<th>Significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Ano de lançamento</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Contagem de jogos lançados no ano</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Média de todas as notas de críticos por ano</td>
</tr>
</tbody>
</table>

In [166]:
%%sql 
SELECT 
  year, 
  avg_critic_score 
FROM 
  top_critic_years 
EXCEPT 
SELECT 
  year, 
  avg_critic_score 
FROM 
  top_critic_years_more_than_four_games 
ORDER BY 
  avg_critic_score DESC;
;


 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


### 6. Anos com melhores avaliações de usuários
Baseado nas consultas anteriores os anos 90 talvez sejam considerados a era de ouro dos video games, mas é necessário levar em conta a avaliação dos jogadores.

In [168]:
%%sql 
SELECT 
  g.year, 
  COUNT(g.game) AS num_games, 
  ROUND(
    AVG(r.user_score), 
    2
  ) AS avg_user_score 
FROM 
  game_sales g 
  INNER JOIN reviews r ON g.game = r.game 
GROUP BY 
  g.year 
HAVING 
  COUNT(g.game) > 4 
ORDER BY 
  avg_user_score DESC 
LIMIT 
  10;


 * postgresql:///games
10 rows affected.


year,num_games,avg_user_score
1997,8,9.50
1998,10,9.40
2010,23,9.24
2009,20,9.18
2008,20,9.03
1996,5,9.00
2005,13,8.95
2006,16,8.95
2000,8,8.80
2002,9,8.80


### 7. Anos amados por críticos e jogadores
Foram criadas duas tabelas com os valores das consultas anteriores</p>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>Tipo</th>
<th>Significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Ano de lançamento</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Contagem de jogos lançados no ano</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Média de todas as notas de críticos por ano</td>
</tr>
</tbody>
</table>
<p>We've also saved the results of our top user years query from the previous task into a table:</p>
<h3 id="top_user_years_more_than_four_games"><code>top_user_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>Tipo </th>
<th>Significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Ano de lançamento</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Número de lançamentos</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_user_score</code></td>
<td>float</td>
<td>Média de todas as notas de usúarios por ano</td>
</tr>
</tbody>
</table>

In [170]:
%%sql 
SELECT year
FROM top_user_years_more_than_four_games
INTERSECT
SELECT year
FROM top_critic_years_more_than_four_games;

 * postgresql:///games
3 rows affected.


year
1998
2008
2002


## 8. Vendas nos melhores anos dos video games
Quantos jogos foram vendidos nos anos mais bem avaliados por críticos e jogadores

In [172]:
%%sql 
SELECT g.year, SUM(g.games_sold) AS total_games_sold
FROM game_sales g
WHERE g.year IN (SELECT year
FROM top_user_years_more_than_four_games
INTERSECT
SELECT year
FROM top_critic_years_more_than_four_games)
GROUP BY g.year
ORDER BY total_games_sold DESC;


 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67
